In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchsummary import summary

In [2]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

])

In [3]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform = transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform = transform)

100%|██████████| 170M/170M [00:03<00:00, 48.5MB/s]


In [4]:
batch_size = 128
num_class = 10
epochs = 5
learning_rate = 0.001
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [14]:
# Load pre-trained SqueezeNet1.0
model = models.squeezenet1_0(pretrained=True)

# Freeze feature extractor layers
for param in model.features.parameters():
    param.requires_grad = False

# Save original final conv layer (optional)
final_conv = model.classifier[1]

# Replace final conv layer to match number of classes
model.classifier[1] = nn.Conv2d(512, num_class, kernel_size=(1, 1), stride=(1, 1))

# Adjust average pooling (kernel size depends on input image size)
model.classifier[3] = nn.AvgPool2d(kernel_size=13, stride=1)

# Custom forward function
def squeezenet_forward(self, x):
    x = self.features(x)        # Feature extraction
    x = self.classifier(x)      # Classification
    return torch.flatten(x, 1)  # Flatten output





In [12]:
print(model.classifier)

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Conv2d(512, 10, kernel_size=(1, 1), stride=(1, 1))
  (2): ReLU(inplace=True)
  (3): AvgPool2d(kernel_size=13, stride=1, padding=0)
)


In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
summary(model, (3,224, 224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 109, 109]          14,208
              ReLU-2         [-1, 96, 109, 109]               0
         MaxPool2d-3           [-1, 96, 54, 54]               0
            Conv2d-4           [-1, 16, 54, 54]           1,552
              ReLU-5           [-1, 16, 54, 54]               0
            Conv2d-6           [-1, 64, 54, 54]           1,088
              ReLU-7           [-1, 64, 54, 54]               0
            Conv2d-8           [-1, 64, 54, 54]           9,280
              ReLU-9           [-1, 64, 54, 54]               0
             Fire-10          [-1, 128, 54, 54]               0
           Conv2d-11           [-1, 16, 54, 54]           2,064
             ReLU-12           [-1, 16, 54, 54]               0
           Conv2d-13           [-1, 64, 54, 54]           1,088
             ReLU-14           [-1, 64,

In [19]:
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)

citerion = nn.CrossEntropyLoss()

def train_model(model, train_loader, epochs):
  model.train()
  for epoch in range(epochs):
    total_running_loss = 0
    correct = 0
    for input, label in train_loader:
      input, label = input.to(device), label.to(device)
      optimizer.zero_grad()
      output = model(input)
      loss = citerion(output, label)
      loss.backward()
      optimizer.step()
      total_running_loss += loss.item() * input.size(0)
      pred = output.argmax(dim=1)
      correct += pred.eq(label).sum().item()
    avg_loss = total_running_loss / len(train_loader.dataset)
    accuracy = correct / len(train_loader.dataset)
    print(f'Epoch: [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')


train_model(model, train_loader, epochs)

Epoch: [1/5], Loss: 0.8038, Accuracy: 0.7253
Epoch: [2/5], Loss: 0.5649, Accuracy: 0.8058
Epoch: [3/5], Loss: 0.5260, Accuracy: 0.8156
Epoch: [4/5], Loss: 0.5159, Accuracy: 0.8195
Epoch: [5/5], Loss: 0.5011, Accuracy: 0.8262


In [20]:
def evaluate_model (model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for input, label in test_loader:
      input, label = input.to(device), label.to(device)
      output = model(input)
      loss = citerion(output, label)
      test_loss += loss.item() * input.size(0)
      pred = output.argmax(dim=1)
      correct += pred.eq(label).sum().item()
    avg_loss = test_loss / len(test_loader.dataset)
    accuracy  = correct / len(test_loader.dataset)
    print(f'Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

evaluate_model(model, test_loader)

Test Loss: 0.5018, Accuracy: 0.8275
